# Smart meters using Semi Quantum Key Distribution

In [27]:
from qiskit import *
import math
import numpy as np
import hashlib


In [28]:
simulator = Aer.get_backend('qasm_simulator')

## File handling

In [29]:
# path to control centre and edge node database
cc_en_db_path = './database/control_center_edge_node_database.txt'

# path to control centre and smart meter database
cc_sm_db_path = './database/control_center_smart_meter_database.txt'

In [30]:
def addNewDevice(file_path, devicename=None, password=None):
    '''
    file_path(str): path to file on which username and password are stored
    devicename(str): devicename of the new device
    password(str): password of the new device

    return bool: True if device is added else False
    '''

    if file_path == None:
        print("File path None")
        return False
    
    if devicename == None:
        print("Username None")
        return False

    if password == None:
        print("Password None")
        return False

    with open(file_path, "r") as file:
        for device in file.read().split('\n')[:-1]:
            [_devicename, _password] = device.split(':')
            if devicename == _devicename:
                print("Device Already Exists")
                return False

    with open(file_path, "a") as file:
        file.write(f'{devicename}:{password}\n')
        print("Successfully registered")
        return True

In [31]:
def loginDevice(file_path, devicename, password):
    '''
    file_path(str): path to file on which username and password are stored
    devicename(str): devicename of the device
    password(str): password of the device

    return bool: True if device is authenticated else False
    '''

    if devicename == None or password == None or file_path == None:
        return False

    with open(file_path, "r") as file:
        for device in file.read().split('\n')[:-1]:
            [_devicename, _password] = device.split(':')
            if devicename == _devicename and password == _password:
                return True
    return False

In [32]:
def getDeviceKey(file_path, devicename):
    '''
    file_path(str): path to file on which username and password are stored
    devicename(str): devicename of the device

    return str: password is device is found else None 
    '''

    if devicename == None or file_path == None:
        return False

    with open(file_path, "r") as file:
        for device in file.read().split('\n')[:-1]:
            [_devicename, _password] = device.split(':')
            if devicename == _devicename:
                return _password
    return None

## SQKD

In [33]:
# size of INFO BITS
n = 4

# delta parameter for sqkd
delta = 1/8

# (Z-)error threshold for CTRL
p_ctrl = 0.5

# (Z-)error threshold for SIFT
p_test = 0.5

In [34]:
def encodeSQKD(basis, message_bit, circuit, idx):
	'''
	basis(int): defines what basis is the message encode. 0 (Computational) and 1 (Hadamard)
	message_bit (int):
    0 is encoded as |0> or |+>
    1 is encoded as |1> or |->
    circuit(QuantumCircuit): the circuit used for communication
    idx(int): index of the message_bit
	'''
	if message_bit == 1:
		circuit.x(idx)
	if basis == 1:
		circuit.h(idx)
	return circuit

In [35]:
def sqkd(device, devicename):
    '''
    device(str): "SMART_METER" or "EDGE_NODE"
    devicename(str): name of device

    Saves data in database if SQKD is successfull
    
    return bool: True if SQKD is successfull else False
    '''

    # number of bits to be set to device
    N = math.ceil(8*n*(1+delta))

    # generate a message for sqkd (binary array)
    message = np.random.randint(2, size=N)
    print(f'{message=}')

    # generate basis for encoding qubits by control center (alice)
    cc_basis = np.random.randint(2, size=N)
    print(f'{cc_basis=}')

    # circuit for sending data to device
    circuit = QuantumCircuit(N, N)

    for (idx, (basis, message_bit)) in enumerate(zip(cc_basis, message)):
        circuit = encodeSQKD(basis, message_bit, circuit, idx)

    circuit.barrier()

    # device basis for SIFT or CTRL
    device_decisions = np.random.randint(2, size=N)
    print(f'{device_decisions=}')

    # get the reflected (CTRL) and measured (SIFT) qubits

    ctrl_qubits = []

    for (idx, decesion) in enumerate(device_decisions):
        if decesion == 0:
            ctrl_qubits.append(idx)

    print(f'{ctrl_qubits=}')
    sift_qubits = []

    for (idx, decesion) in enumerate(device_decisions):
        if decesion == 1:
            sift_qubits.append(idx)

    print(f'{sift_qubits=}')

    z_sift = [sift_qubits[i] for i in range(len(sift_qubits)) if cc_basis[sift_qubits[i]] == 0]

    print(f'{z_sift=}')
    # SIFT by device

    for idx in sift_qubits:
        circuit.measure(idx, idx)
    # measurement of sifted qubits by device

    result_device = execute(circuit, backend=simulator, shots=1).result()
    count_device = result_device.get_counts(circuit)

    device_measured_value = list(count_device.keys())[0][::-1]
    
    circuit.barrier()


    print(f'{device_measured_value=}')

    ######### PENDING ####################
    # reorder the reflected (CTRL) qubits

    ###################################

    # measurement of reflected qubits by control centre

    for idx in ctrl_qubits:
        if cc_basis[idx] == 1:
            circuit.h(idx)
        circuit.measure(idx, idx)

    result_ctrl = execute(circuit, backend=simulator, shots=1).result()
    count_ctrl = result_ctrl.get_counts(circuit)

    ctrl_measured_value = list(count_ctrl.keys())[0][::-1]

    print(f'{ctrl_measured_value=}')

    # calculating error for ctrl

    z_error_ctrl = 0
    x_error_ctrl = 0
    len_z = 0
    len_x = 0

    for idx in ctrl_qubits:
        if cc_basis[idx] == 0:
            if message[idx] != int(ctrl_measured_value[idx]):
                print("z")
                print(idx)
                z_error_ctrl += 1
            len_z += 1
        elif cc_basis[idx] == 1:
            if message[idx] != int(ctrl_measured_value[idx]):
                print("x")
                print(idx)
                x_error_ctrl += 1
            len_x += 1

    print(f'{z_error_ctrl/len_z=}')
    print(f'{x_error_ctrl/len_x=}')

    if z_error_ctrl/len_z < p_ctrl and x_error_ctrl/len_x < p_ctrl:

        # select n random sift bits in z basis as test bits

        test_bits = set()
        while len(test_bits) < n:
            test_bits = set(np.random.choice(z_sift, size=n))
        
        print(f'{test_bits=}')

        # defining remaning string

        v = []

        for idx in z_sift:
            if idx not in test_bits:
                v.append(idx)

        print(f'{v=}')
        # calculating z error for sift

        z_error_test = 0

        for idx in test_bits:
            if int(device_measured_value[idx]) != int(ctrl_measured_value[idx]):
                z_error_test += 1

        print(f'{z_error_test/len(test_bits)=}')

        if z_error_test/len(test_bits) < p_test and len(v) >= n:

            info_bits = v[:n]

            info = ""

            for idx in info_bits:
                info += device_measured_value[idx]

            print(f'{info=}')

            # Apply hash to function to info
            sk = hashlib.sha512(info.encode('utf-8')).hexdigest()

            print(f'{sk=}')

            ######## SAVE IN FILE ###############

            file_path = None

            if device == "SMART_METER":
                file_path = cc_sm_db_path

            elif device == "EDGE_NODE":
                file_path = cc_en_db_path


            if addNewDevice(file_path, devicename, info):
                print("KEY SAVED")
                return True
    
    print("SQKD FAILED")
    return False

## Semi-Quantum Mutual Identity Authentication Using Bell States

$$ |\varphi^{\pm}> = \frac{1}{\sqrt{2}}(|00>\pm|11>) (0, 1)$$
$$ |\phi^{\pm}> = \frac{1}{\sqrt{2}}(|01>\pm|10>) (2, 3)$$

In [36]:
def createBellStates(state_number, qubit1, qubit2, circuit):
    '''
    state_number (int): bell state based on above cell
    qubit1 (int), qubit2 (int): indices of entangled qubits
    circuit (QuantumCircuit): circuit under consideration

    return QuantumCircuit: updated quantum circuit
    '''
    if state_number == 0:
        pass
    elif state_number == 1:
        circuit.x(qubit1)
    elif state_number == 2:
        circuit.x(qubit2)
    elif state_number == 3:
        circuit.x(qubit1)
        circuit.x(qubit2)
    circuit.h(qubit1)
    circuit.cnot(qubit1, qubit2)
    return circuit

In [37]:
def createDecoyQubits(key, bit, idx, circuit):
    '''
    key (str): securely shared key
    bit (int): 0 or 1
    idx (int): index of the decoy bit
    circuit (QuantumCircuit): circuit under consideration

    return QuantumCircuit: updated quantum circuit
    '''
    if bit:
        circuit.x(idx)
    if key[2*idx:2*idx+2] in {"10", "11"}:
        circuit.h(idx)
    return circuit

In [38]:
def sq_auth(device, devicename):
    '''
    device(str): "SMART_METER" or "EDGE_NODE"
    devicename(str): devicename of the device

    return (bool bool): (is classical entity legal, is quantum entity legal)
    '''

    # get file

    file_path = None

    if device == "SMART_METER":
        file_path = cc_sm_db_path

    elif device == "EDGE_NODE":
        file_path = cc_en_db_path

    # get key

    key = getDeviceKey(file_path, devicename)

    if key == None:
        print("Device does not exists")
        return (False, False)

    print(key)

    n_prime = n//2  # since size of the key in paper is 2*n
    # we define n' as 2*n' = n

    # STEP 1: Preparation

    circuit = QuantumCircuit(n_prime*2, n_prime*2)

    # Even bits = S_H
    # Odd bits = S_T

    initial_bell_state_number = list(np.random.randint(0, 4, size=(n_prime)))
    print(f'{initial_bell_state_number=}')

    # n_prime*2 for n bell states

    # generating random n' bell states
    for i in range(n_prime):
        circuit = createBellStates(initial_bell_state_number[i], 2*i, 2*i+1, circuit)

    # STEP 2: Eavesdropping detection

    ## generating n'/2 decoy qubits

    decoy_circuit = QuantumCircuit(n_prime//2, n_prime//2)

    decoy_init = np.random.randint(0, 2, size=(n_prime//2))

    print(f'{decoy_init=}')

    for i in range(n_prime//2):
        decoy_circuit = createDecoyQubits(key, decoy_init[i], i, decoy_circuit)

    ###### Pending ######
    # reordering
    #####################

    for i in range(n_prime//2):
        if key[2*i:2*i+2] in {"10", "11"}:
            decoy_circuit.h(i)
        decoy_circuit.measure(i, i)
    
    decoy_result = execute(decoy_circuit, backend=simulator, shots=1).result()
    decoy_count = decoy_result.get_counts(decoy_circuit)
    decoy_measured_values = [int(b) for b in list(decoy_count.keys())[0][::-1]]

    print(f'{decoy_measured_values=}')

    for i in range(n_prime//2):
        if decoy_measured_values[i] != decoy_init:
            print("AUTH FAILED")
            return (False, False)

    # STEP 3: Measurement

    for i in range(n_prime*2):
        circuit.measure(i, i)

    result = execute(circuit, backend=simulator, shots=1).result()
    count = result.get_counts(circuit)
    measured_values = [int(b) for b in list(count.keys())[0][::-1]]

    # STEP 4: XOR operation

    ## Create RB

    RB = []

    for i in range(1, n, 2):
        RB.append(measured_values[i])
    print(f'{RB=}')
    
    ## Create IA

    RA = []

    for i in range(0, n, 2):
        RA.append(measured_values[i])
    print(f'{RA=}')

    ## Create RA*

    RAstar = []

    for i in range(n_prime):
        if RB[i] == 0:
            if initial_bell_state_number[i] in {0, 1}:
                RAstar.append(0)
            else:
                RAstar.append(1)
        elif RB[i] == 1:
            if initial_bell_state_number[i]  in {0, 1}:
                RAstar.append(1)
            else:
                RAstar.append(0)
    print(f'{RAstar=}')


    ## Create RB*

    RBstar = []

    for i in range(n_prime):
        if RA[i] == 0:
            if initial_bell_state_number[i]  in {0, 1}:
                RBstar.append(0)
            else:
                RBstar.append(1)
        elif RA[i] == 1:
            if initial_bell_state_number[i]  in {0, 1}:
                RBstar.append(1)
            else:
                RBstar.append(0)
    print(f'{RBstar=}')

    ## Create IA, IAstar, IB, IBstar
    IA = []
    IB = []
    IAstar = []
    IBstar = []

    for i in range(n_prime):
        IA.append(RA[i]^int(key[i]))
        IB.append(RB[i]^int(key[i]))
        IAstar.append(RAstar[i]^int(key[i]))
        IBstar.append(RBstar[i]^int(key[i]))

    print(f'{IA=}')
    print(f'{IB=}')
    print(f'{IAstar=}')
    print(f'{IBstar=}')

    # STEP 5: Authentication

    bob_auth, alice_auth = 1, 1

    for i in range(n_prime):
        if IA[i] != IAstar[i]:
            alice_auth = 0
        if IB[i] != IBstar[i]:
            bob_auth = 0
    
    print(f'{measured_values=}')

    # print(circuit.draw())
    # print(decoy_circuit.draw())

    print(f'{(bob_auth, alice_auth)=}')

    return (bob_auth, alice_auth)

## Test cells

In [41]:
smart_meter_devicename = "sm1"
reg = None
reg = sqkd("SMART_METER", smart_meter_devicename)
if reg:
    sm_auth, cc_auth = sq_auth("SMART_METER", smart_meter_devicename)

message=array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1])
cc_basis=array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0])
device_decisions=array([1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0])
ctrl_qubits=[3, 4, 6, 8, 10, 12, 15, 16, 17, 19, 23, 26, 29, 30, 33, 35]
sift_qubits=[0, 1, 2, 5, 7, 9, 11, 13, 14, 18, 20, 21, 22, 24, 25, 27, 28, 31, 32, 34]
z_sift=[5, 7, 9, 13, 14, 18, 20, 21, 24, 27, 34]
device_measured_value='110000010001011000100010110000000010'
ctrl_measured_value='111000010010011111100001110011010111'
z_error_ctrl/len_z=0.0
x_error_ctrl/len_x=0.0
test_bits={9, 34, 27, 7}
v=[5, 13, 14, 18, 20, 21, 24]
z_error_test/len(test_bits)=0.0
info='0111'
sk='31ef4e54dd72993d2cb4a16433b58220d5a91a1510a78b7534469c3242d12f75950574e50d3c9e62e9ba5c9f025e377fb5b29f963254e43cf7009f5